# Datenanalyse - Balance

##### Initialisierung

In [18]:
%load_ext autoreload
%autoreload 2

import src.config
from src.data_loader import get_interchange_data
import plotly.express as px
import polars as pl

df = get_interchange_data()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### Map Overview

In [19]:
import plotly.graph_objects as go
import polars as pl

def plot_energy_network(df_interchange, coords_map):    
    region_map = {
        "MIDW": ["IL","IN","MI","WI","OH","MN","IA","MO"],
        "NW": ["WA","OR","ID","MT","WY"],
        "SW": ["AZ","NM","NV","UT","CO"],
        "CAL": ["CA"],
        "CAR": ["NC","SC"],
        "TEX": ["TX"],
        "FLA": ["FL"],
        "NE": ["ME","NH","VT","MA","RI","CT"],
        "NY": ["NY"],
        "MIDA": ["NJ","PA","DE","MD","DC","VA","WV"],
        "SE": ["GA","AL","MS","TN","KY","AR","LA"],
        "CENT": ["KS","NE","SD","ND","OK"],
        "TEN": ["TN"],
    }

    # 1. Farbschema für die Regionen
    region_colors = {
        "MIDW": "#1f77b4", "NW": "#2ca02c", "SW": "#d62728", 
        "CAL": "#9467bd", "CAR": "#8c564b", "TEX": "#e377c2", 
        "FLA": "#7f7f7f", "NE": "#bcbd22", "NY": "#17becf", 
        "MIDA": "#aec7e8", "SE": "#ff7f0e", "CENT": "#ffbb78", "TEN": "#98df8a"
    }

    fig = go.Figure()

    # 2. CHOROPLETH LAYER: Flächen einfärben
    # Wir müssen die region_map "umdrehen": State -> Region
    state_to_region = {state: reg for reg, states in region_map.items() for state in states}
    all_states = list(state_to_region.keys())
    
    # Numerische Werte für die Farbskala (Plotly braucht Zahlen für Choropleth-Farben)
    region_index = {reg: i for i, reg in enumerate(region_colors.keys())}
    z_values = [region_index[state_to_region[st]] for st in all_states]
    
    # Custom Colorscale erstellen
    colorscale = [[i/(len(region_colors)-1), color] for i, color in enumerate(region_colors.values())]

    fig.add_trace(go.Choropleth(
        locations=all_states,
        z=z_values,
        locationmode='USA-states',
        colorscale=colorscale,
        showscale=False,
        marker_line_color='white', # Grenzen zwischen den Staaten
        marker_line_width=0.5,
        hoverinfo='location'
    ))

    # 3. VERBINDUNGEN (Directed Flows)
    connections = df_interchange.group_by(["balancing_authority", "directly_interconnected_balancing_authority"]).agg(
        pl.col("interchange_mw").mean().alias("avg_flow")
    )

    for conn in connections.iter_rows(named=True):
        ba, diba, flow = conn["balancing_authority"], conn["directly_interconnected_balancing_authority"], conn["avg_flow"]
        if ba in coords_map and diba in coords_map and abs(flow) > 5:
            start, end = (coords_map[ba], coords_map[diba]) if flow > 0 else (coords_map[diba], coords_map[ba])
            fig.add_trace(go.Scattergeo(
                lon=[start[1], end[1], None], lat=[start[0], end[0], None],
                mode='lines',
                line=dict(width=max(0.8, min(abs(flow)/300, 5)), color='rgba(50, 50, 50, 0.6)'),
                showlegend=False
            ))

    # 4. KNOTEN (BAs)
    # Wir nutzen das ba_to_region Mapping für die Punktfarbe
    ba_to_region = dict(df_interchange.select(["balancing_authority", "region"]).unique().iter_rows())

    for region_name, color in region_colors.items():
        bas_in_this_region = [ba for ba, reg in ba_to_region.items() if reg == region_name and ba in coords_map]
        if not bas_in_this_region: continue
            
        fig.add_trace(go.Scattergeo(
            lon=[coords_map[ba][1] for ba in bas_in_this_region],
            lat=[coords_map[ba][0] for ba in bas_in_this_region],
            mode='markers+text',
            textposition="top center",
            name=region_name,
            marker=dict(size=10, color=color, line=dict(width=1.5, color='white')),
            text=bas_in_this_region,
            hovertext=[f"Authority: {ba}<br>Region: {region_name}" for ba in bas_in_this_region],
            hoverinfo="text"
        ))

    # 5. LAYOUT
    fig.update_layout(
        title_text='US Energy Grid: Regional Choropleth & Connectivity',
        geo=dict(
            scope='usa', # 'usa' zeigt automatisch auch Teile von Kanada/Mexiko bei mercator
            projection_type='albers usa',
            showcountries=True
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        height=800
    )

    return fig


coords_map = {
    "PJM": [40.07, -75.30],   # PJM Interconnection (Valley Forge, PA)
    "NYIS": [42.65, -73.75],  # New York ISO (Rensselaer/Albany, NY)
    "AECI": [38.57, -92.17],  # Associated Electric Cooperative (Jefferson City, MO)
    "GCPD": [47.50, -120.50], # Grant County PUD (Ephrata, WA)
    "AZPS": [33.45, -112.07], # Arizona Public Service (Phoenix, AZ)
    "SCEG": [34.00, -81.03],  # Dominion Energy SC (Columbia, SC)
    "JEA": [30.33, -81.66],   # Jacksonville Electric Authority (Jacksonville, FL)
    "IPCO": [43.62, -116.20], # Idaho Power (Boise, ID)
    "DUK": [35.23, -80.84],   # Duke Energy (Charlotte, NC)
    "MISO": [41.88, -87.63],  # Midcontinent ISO (Chicago, IL)
    "TEPC": [32.22, -110.97], # Tucson Electric Power (Tucson, AZ)
    "TVA": [35.05, -85.31],   # Tennessee Valley Authority (Chattanooga, TN)
    "PACW": [45.52, -122.67], # PacifiCorp West (Portland, OR)
    "PSEI": [47.61, -122.33], # Puget Sound Energy (Bellevue/Seattle, WA)
    "SEPA": [33.75, -84.39],  # Southeastern Power Administration (Atlanta, GA)
    "WAUW": [44.96, -89.63],  # Wausau area utility region (Wausau, WI)
    "AVA": [47.66, -117.42],  # Avista (Spokane, WA)
    "CPLW": [35.78, -78.64],  # Duke Energy Progress West (Raleigh, NC)
    "WWA": [47.25, -122.44],  # Tacoma Power (Tacoma, WA)
    "ISNE": [42.36, -71.06],  # ISO New England (Holyoke/Boston region)
    "PNM": [35.08, -106.65],  # Public Service Co. of NM (Albuquerque, NM)
    "BANC": [38.58, -121.49], # Balancing Authority of Northern California (Sacramento, CA)
    "PGE": [45.52, -122.67],  # Portland General Electric (Portland, OR)
    "GRID": [51.50, -0.12],   # National Grid placeholder (London HQ)
    "IID": [32.79, -115.56],  # Imperial Irrigation District (El Centro, CA)
    "DOPD": [47.66, -118.74], # Douglas County PUD (East Wenatchee, WA)
    "SEC": [32.54, -83.90],   # Seminole Electric Cooperative (Tucker GA region)
    "FPC": [28.54, -81.38],   # Duke Energy Florida (Orlando, FL)
    "NWMT": [46.59, -112.04], # NorthWestern Energy (Helena, MT)
    "LDWP": [34.05, -118.24], # Los Angeles Dept. of Water & Power (LA, CA)
    "TPWR": [47.25, -122.44], # Tacoma Power (Tacoma, WA)
    "PSCO": [39.74, -104.99], # Public Service Co. of Colorado (Denver, CO)
    "SCL": [47.61, -122.33],  # Seattle City Light (Seattle, WA)
    "SIKE": [47.49, -117.59], # Inland Power & Light (Spokane region)
    "TIDC": [37.64, -120.99], # Turlock Irrigation District (Turlock, CA)
    "HST": [25.47, -80.48],   # City of Homestead (Homestead, FL)
    "DEAA": [64.84, -147.72], # Golden Valley Electric (Fairbanks, AK)
    "EPE": [31.76, -106.49],  # El Paso Electric (El Paso, TX)
    "SC": [33.92, -81.03],    # Santee Cooper (Moncks Corner, SC)
    "WACM": [44.08, -103.23], # Western Area Colorado-Missouri (Rapid City, SD)
    "CHPD": [47.84, -120.02], # Chelan County PUD (Wenatchee, WA)
    "GWA": [47.66, -117.42],  # Grant/WA aggregation placeholder
    "ERCO": [30.27, -97.74],  # ERCOT (Austin, TX)
    "PACE": [40.76, -111.89], # PacifiCorp East (Salt Lake City, UT)
    "LGEE": [38.25, -85.76],  # LG&E and KU (Louisville, KY)
    "SOCO": [33.52, -86.80],  # Southern Company (Birmingham, AL)
    "SWPP": [38.63, -90.20],  # Southwest Power Pool (Little Rock, AR region)
    "AVRN": [45.52, -122.67], # Avangrid NW placeholder
    "FMPP": [28.54, -81.38],  # Florida Municipal Power Pool (Orlando, FL)
    "WALC": [33.45, -112.07], # Western Area Lower Colorado (Phoenix, AZ)
    "BPAT": [45.64, -122.66], # Bonneville Power Administration (Portland, OR)
    "SPA": [34.75, -92.29],   # Southwestern Power Administration (Little Rock, AR)
    "CISO": [38.58, -121.49], # California ISO (Folsom/Sacramento, CA)
    "NEVP": [36.17, -115.14], # NV Energy (Las Vegas, NV)
    "YAD": [35.80, -80.26],   # Alcoa Yadkin Division (NC)
    "CPLE": [35.23, -80.84],  # Duke Energy Progress East (Charlotte, NC)
    "FPL": [25.76, -80.19],   # Florida Power & Light (Miami, FL)
    "TEC": [27.95, -82.46],   # Tampa Electric (Tampa, FL)
    "SRP": [33.45, -112.07],  # Salt River Project (Phoenix, AZ)
    "GVL": [29.65, -82.32],   # Gainesville Regional Utilities (Gainesville, FL)
    "TAL": [30.44, -84.28],   # City of Tallahassee Utilities (Tallahassee, FL)
}

fig = plot_energy_network(df, coords_map)

fig.show()